# 📈 CrewAI Multi-Agent Financial Analysis System

This project demonstrates a multi-agent system for analyzing financial markets using [CrewAI](https://github.com/joaomdmoura/crewai), OpenAI's GPT-3.5 Turbo, and real-time web tools.

It simulates a team of collaborative agents that:
- 🔍 **Monitor live stock data**
- 🧠 **Generate trading strategies**
- 🛠 **Plan trade executions**
- ⚠️ **Evaluate financial risk**

## 🚀 Technologies Used
- 🧠 OpenAI GPT-3.5 Turbo (via `langchain-openai`)
- 🕸 CrewAI multi-agent framework
- 🔧 Serper.dev (Google Search API)
- 📄 Python + Jupyter Notebook

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from langchain_openai import ChatOpenAI
import os

from utils import get_openai_api_key, get_serper_api_key

In [ ]:
openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [ ]:
# Tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [ ]:
# Create Agents
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent uses statistical modeling and machine learning to provide crucial insights.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool]
)

trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based on insights from the Data Analyst Agent.",
    backstory="This agent devises and refines trading strategies based on financial data and market trends.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool]
)

execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies based on approved trading strategies.",
    backstory="Specializing in timing and logistics of trades to optimize results.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool]
)

risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks associated with potential trading activities.",
    backstory="Expert in risk assessment models and financial safeguards.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool]
)

In [ ]:
# Create Tasks
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent
)

# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent
)

# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the best execution methods "
        "for {stock_selection}, considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to execute trades for {stock_selection}."
    ),
    agent=execution_agent
)

# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading strategies and execution plans for "
        "{stock_selection}. Provide a detailed analysis of potential risks and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent
)

In [ ]:
# Create the crew
# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[
        data_analyst_agent,
        trading_strategy_agent,
        execution_agent,
        risk_management_agent
    ],
    tasks=[
        data_analysis_task,
        strategy_development_task,
        execution_planning_task,
        risk_assessment_task
    ],
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

In [ ]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [ ]:
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

In [ ]:
from IPython.display import Markdown
Markdown(result)